<div style="overflow: hidden;">
    <img src="images/DREGS_logo_v2.png" width="300" style="float: left; margin-right: 10px;">
</div>

# Getting started: Part 1 - Registering datasets

The *DESC data registry* is a means of storing and keeping track of DESC related datasets, i,e., where they are, when they were produced and what precursor datasets they depend on.

This is a quick tutorial to get your started using the `dataregistry` at NERSC, both for entering your own datasets into the registry, and for finding out what datasets already exist through queries.

### What we cover in this tutorial

In this tutorial we will learn how to:

1) Connect to the DESC data registry using the `DataRegistry` class
2) Register a dataset
3) Update a registered dataset with a new version

### Before we begin

If you haven't done so already, check out the [getting setup](https://lsstdesc.org/dataregistry/tutorial_setup.html) page from the documentation if you want to run this tutorial interactively.

A quick way to check everything is set up correctly is to run the first cell below, which should load the `dataregistry` package, and print the package version.

In [ ]:
import dataregistry
print("Working with dataregistry version:", dataregistry.__version__)

## 1) The DataRegistry class

The top-level `DataRegistry` class provides a quick and easy-to-use interface to `dataregistry` functionality.

In many cases, in particular when you only need query functionality, or when you're working on your own at NERSC, a simple call with no additional arguments is adequate, e.g.,

In [ ]:
from dataregistry import DataRegistry

# Establish connection to database (using defaults)
datareg = DataRegistry()

With no arguments, the `DataRegistry` class will automatically attempt to:
- establish a connection to the registry database using the information in your `~/.config_reg_access` and `~/.pgpass` files
- connect to the default database schema
- use the default NERSC "`site`" for the `root_dir`

The root directory (`root_dir`) is the base path where all ingested data will be copied to. Other than for testing, this should generally be the NERSC `site` address.

### Special cases away from the defaults

If you are not connecting to the default database schema, or if your configuration file is located somewhere other than your `$HOME` directory, you must provide that information during the object creation, e.g.,

In [ ]:
# When your configuration file is not in the default location
# datareg = DataRegistry(config_file="/path/to/config")

# If you are connecting to a database schema other than the default, you need to specify the schema name to connect to
# datareg = DataRegistry(schema="myschema")

# If you want to specify the root_dir that data is copied to (this should only be changed for testing purposes)
# datareg = DataRegistry(root_dir="/my/root/dir")

### Setting universal `owner`'s and/or `owner_type`s for datasets

When creating a `DataRegistry` class instance which you intend to use for registering datasets, you may optionally set a default `owner` and/or `owner_type` that will be inherited for all datasets that are registered during that instance. See details in the section "*Registering new datasets with DataRegistry*" below for details about `owner` and `owner_type`.

In [ ]:
# Setting a global owner and owner_type default value for all datasets that will be registered during this instance
# datareg = DataRegistry(owner="desc", owner_type="group")

## 2) Registering new datasets with the `DataRegistry` class

Now that we have made our connection to the database, we can register some datasets using the `Registrar` extension of the `DataRegistry` class.

In [ ]:
# Create an empty text file as some example data
with open("dummy_dataset.txt", "w") as f:
    f.write("some data")

# Add new entry.
dataset_id, execution_id = datareg.Registrar.dataset.register(
    "nersc_tutorial/my_desc_dataset",
    "1.0.0",
    description="An output from some DESC code",
    owner="DESC",
    owner_type="group",
    is_overwritable=True,
    old_location="dummy_dataset.txt"
)

# This is the unique identifier assigned to the dataset from the registry
print(f"Dataset {dataset_id} created")

# This is the id of the execution the dataset belongs to (see next tutorial)
print(f"Dataset assigned to execution {execution_id}")

This will register a new dataset. A few notes:

### The relative path

Datasets are registered within the registry under a path relative to the root directory (`root_dir`), which, by default, is a shared space at NERSC. For those interested, the eventual full path for the dataset will be `<root_dir>/<schema>/<owner_type>/<owner>/<relative_path>`. This means that the combination of `relative_path`, `owner` and `owner_type` must be unique within the registry, and therefore cannot already be taken when you register a new dataset (an exception to this is if you allow your datasets to be overwritable, see below). 

The relative path is one of the two required parameters you must specify when registering a dataset (in the example here our relative path is `nersc_tutorial/my_desc_dataset`).

### The version string

The second required parameter is the version string, in the semantic format, i.e., MAJOR.MINOR.PATCH. There exists also an optional ``version_suffix`` parameter, which may be used to further identify the dataset, e.g. with a value like "rc1" to make it clear it's only a release candidate, possibly not in its final form.


### Owner and Owner type

Datasets are registered under a given `owner`. This can be any string, however it should be informative. If no `owner` is specified, and no global `owner` was set when the `DataRegistry` instance was created, `$USER` is used as the default.

One further level of classification is the `owner_type`, which can be one of `user`, `group`, `project` or `production`. If `owner_type` is not specified, and no global `owner_type` was set when the `DataRegistry` instance was created, `user` is the default.

Note that `owner_type="production"` datasets can only go into the production schema, and can never be overwritten (see the "[Production Schema](https://github.com/LSSTDESC/dataregistry/tree/main/docs/source/tutorial_notebooks/production_scheme.ipynb)" tutorial for more information).

### Overwriting datasets

By default, datasets in the data registry, once registered, are not overwritable. You can change this behavior by setting `is_overwritable=True` when registering your datasets. If `is_overwritable=True` on one of your previous datasets, you can register a new dataset with the same combination of `relative_path`, `owner` and `owner_type` as before (be warned that any previous data stored under this path will be deleted first). 

Note that whilst the data in the shared space will be overwritten with each registration when `is_overwritable=True`, the original entries in the data registry database are never lost (a new unique entry is created each time, and the 'old' entries will obtain `True` for their `is_overwritten` row).

### Copying the data

Registering a dataset does two things; it creates an entry in the DESC data registry database with the appropriate metadata, and it (optionally) copies the dataset contents to the `root_dir`. 

If the data are already at the correct relative path within the `root_dir`, then only the relative path needs to be provided, and the dataset will then be registered. However it's likely for most users the data will need to be copied from another location to the `root_dir`. That initial location may be specified using the `old_location` parameter. 

In our example we have created a dummy text file as our dataset and ingested it into the data registry, however this can be any file or directory (directories will be recursively copied).

### Extra options

All the `Registrar.dataset.register()` parameters we do not explicitly specify revert to their default values. For a full list of these options see the documentation [here](http://lsstdesc.org/dataregistry/reference_python.html#the-dregs-class). We would advise you to be as precise as possible when creating entries within the data registry. 

## 3) Updating a previously registered dataset with a newer version

If you have a dataset that has been previously registered within the data registry, and that dataset has updates, you have three options for how to handle the new entry:

- You can enter it as a completely new standalone dataset with no links to the previous dataset
- You can overwrite the existing dataset with the new data (only if the previous dataset was entered with `is_overwritable=True`)
- You can enter it as a new version of the previous dataset (recommended)

Unless you are overwriting a previous dataset (when `is_overwritable=True`), you cannot enter a new dataset (even an updated version) using the same relative path. However, datasets can share the same `name` field, which acts as a reference for the dataset, and which is what we'll use to keep our updated dataset connected to our previous one.

Note that for our test dataset entry above we did not specify a `name` during registration. The default `name` for a dataset is the file or directory name, extracted from the relative path. In our example above this was `my_desc_dataset`. If you are unsure of what name you specified for your previous dataset that you want to overwrite, you will need to query the database first to find out (see next tutorial for queries).

The combination of `name`, `version` and `version_suffix` for any dataset must be unique. As we are updating a dataset with the same name, we have to make sure to update the version to a new value. One handy feature is automatic version "bumping" for datasets, i.e., rather than specifying a new version string manually, one can select "major", "minor" or "patch" for the version string to automatically bump that property of the version string up. In our case, selecting "minor" will automatically generate the version "1.1.0".

In [ ]:
# Add new entry for an updated dataset with an updated version.
updated_dataset_id, updated_execution_id = datareg.Registrar.dataset.register(
    "nersc_tutorial/my_updated_desc_dataset",
    "minor", # Automatically bumps to "1.1.0"
    description="An output from some DESC code (updated)",
    is_overwritable=True,
    old_location="dummy_dataset.txt",
    name="my_desc_dataset" # Using this name links it to the previous dataset.
)

# This is the unique identifier assigned to the updated dataset from the registry
print(f"Dataset {dataset_id} created")

# This is the id of the execution the updated dataset belongs to (see next tutorial)
print(f"Dataset assigned to execution {execution_id}")